<a href="https://colab.research.google.com/github/Abhi23run/CSE8803_DLT_Project/blob/main/CodeBase/DLT_Project_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install -q -U bitsandbytes
# !pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install transformers==4.31
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets
!pip install evaluate
!pip install -qqq trl==0.7.1
!pip install torch

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
%cd /content/drive/MyDrive/CSE8803_DLT_Project/

/content/drive/MyDrive/CSE8803_DLT_Project


In [26]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import transformers
from torch.optim import Adam
from tqdm import tqdm
from ashish_test import utils
from utils import *
import json
import pandas as pd
import torch
# from torch.utils.data import Dataset, DataLoader
import time
import evaluate
from datasets import Dataset, load_dataset
import random

In [27]:
dir_path = '/content/drive/MyDrive/finqa_dataset'
os.chdir(dir_path)

In [28]:
with open(f"{dir_path}/train.json") as input_file:
        train_data = json.load(input_file)

with open(f"{dir_path}/dev.json") as input_file:
        valid_data = json.load(input_file)

with open(f"{dir_path}/test.json") as input_file:
        test_data = json.load(input_file)

In [29]:
assert len(train_data) == 6251, "The train data seems to be off, please check"
assert len(valid_data) == 883, "The validation data seems to be off, please check"
assert len(test_data) == 1147, "The test data seems to be off, please check"

Loading Ground Truth Serialized Dataset

In [13]:
df_serialized=pd.read_csv(f"{dir_path}/df_combined_table_ser.csv")

In [16]:
df_serialized.columns

Index(['pre_text', 'post_text', 'filename', 'table_ori', 'table', 'id',
       'dataset'],
      dtype='object')

In [21]:
df_serialized.shape

(8281, 7)

In [ ]:
df_serialized[df_serialized['dataset']=='train'][['id','']]

In [20]:
train_data[0].keys()

dict_keys(['pre_text', 'post_text', 'filename', 'table_ori', 'table', 'qa', 'id', 'table_retrieved', 'text_retrieved', 'table_retrieved_all', 'text_retrieved_all'])

In [22]:
train_data[0]['qa']

{'question': 'what is the the interest expense in 2009?',
 'answer': '380',
 'explanation': '',
 'ann_table_rows': [],
 'ann_text_rows': [1],
 'steps': [{'op': 'divide1-1', 'arg1': '100', 'arg2': '100', 'res': '1%'},
  {'op': 'divide1-2', 'arg1': '3.8', 'arg2': '#0', 'res': '380'}],
 'program': 'divide(100, 100), divide(3.8, #0)',
 'gold_inds': {'text_1': 'if libor changes by 100 basis points , our annual interest expense would change by $ 3.8 million .'},
 'exe_ans': 3.8,
 'tfidftopn': {'text_14': 'dollar , would have on the fair value of our forward exchange contracts as of october 31 , 2009 and november 1 , 2008: .',
  'text_0': 'interest rate to a variable interest rate based on the three-month libor plus 2.05% ( 2.05 % ) ( 2.34% ( 2.34 % ) as of october 31 , 2009 ) .'},
 'program_re': 'divide(3.8, divide(100, 100))',
 'model_input': [['text_0',
   'interest rate to a variable interest rate based on the three-month libor plus 2.05% ( 2.05 % ) ( 2.34% ( 2.34 % ) as of october 31 , 2

In [30]:
def prepare_data(json_data, verbose=True):
    net=[]
    err_cnt=0
    for example in json_data:
        question = example["qa"]["question"]
        table = example["table"]
        id=example["id"]
        table_ori=example["table_ori"]
        answer_steps=example["qa"]["steps"]
        table_text = ""
        for row in table[1:]:
            this_sent = table_row_to_text(table[0], row)
            table_text += this_sent
        try:
            steps_text = format_steps(example["qa"]["steps"])
            inputs = {"id":id, "context": table_text, "question": question, "answer": steps_text,\
                      "table":table_ori,"answer_steps":answer_steps}
            net.append(inputs)
        except:
            err_cnt+=1
            if verbose:
                print ("-"*25)
                print (example["filename"])
                print (example["qa"]["steps"])
                print ("-"*25+"\n")
    if err_cnt>0:
        print ("Net Errors:",err_cnt)
    return net

In [31]:
## Preparing huggingface dataset
data_splits = {'train': train_data, 'valid': valid_data, 'test': test_data}
datasets = {split: Dataset.from_pandas(pd.DataFrame(prepare_data(data, False))) for split, data in data_splits.items()}

Net Errors: 198
Net Errors: 35
Net Errors: 39


In [32]:
index=0
datasets['train'][index]

{'id': 'ADI/2009/page_49.pdf-1',
 'context': 'the fair value of forward exchange contracts asset ( liability ) of october 31 2009 is $ 6427 ; the fair value of forward exchange contracts asset ( liability ) of november 1 2008 is $ -23158 ( 23158 ) ;the fair value of forward exchange contracts after a 10% ( 10 % ) unfavorable movement in foreign currency exchange rates asset ( liability ) of october 31 2009 is $ 20132 ; the fair value of forward exchange contracts after a 10% ( 10 % ) unfavorable movement in foreign currency exchange rates asset ( liability ) of november 1 2008 is $ -9457 ( 9457 ) ;the fair value of forward exchange contracts after a 10% ( 10 % ) favorable movement in foreign currency exchange rates liability of october 31 2009 is $ -6781 ( 6781 ) ; the fair value of forward exchange contracts after a 10% ( 10 % ) favorable movement in foreign currency exchange rates liability of november 1 2008 is $ -38294 ( 38294 ) ;',
 'question': 'what is the the interest expense in

In [39]:
step_pattern = r"Step \d+"
import re

In [59]:
def num_steps_dict_fn(example):
    example["num_steps"] = len(re.findall(step_pattern, example["answer"]))
    return example

In [65]:
datasets['train']=datasets['train'].map(num_steps_dict_fn)
datasets['valid']=datasets['valid'].map(num_steps_dict_fn)
datasets['test']=datasets['test'].map(num_steps_dict_fn)

Map:   0%|          | 0/6053 [00:00<?, ? examples/s]

Map:   0%|          | 0/848 [00:00<?, ? examples/s]

Map:   0%|          | 0/1108 [00:00<?, ? examples/s]

In [68]:
datasets['train']

Dataset({
    features: ['id', 'context', 'question', 'answer', 'table', 'answer_steps', 'num_steps'],
    num_rows: 6053
})

**Filtering Datasets for 1 Step**

In [69]:
datasets['train']=datasets['train'].filter(lambda example: example['num_steps']==1)
datasets['valid']=datasets['valid'].filter(lambda example: example['num_steps']==1)
datasets['test']=datasets['test'].filter(lambda example: example['num_steps']==1)

Filter:   0%|          | 0/6053 [00:00<?, ? examples/s]

Filter:   0%|          | 0/848 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1108 [00:00<?, ? examples/s]

In [70]:
datasets

{'train': Dataset({
     features: ['id', 'context', 'question', 'answer', 'table', 'answer_steps', 'num_steps'],
     num_rows: 3532
 }),
 'valid': Dataset({
     features: ['id', 'context', 'question', 'answer', 'table', 'answer_steps', 'num_steps'],
     num_rows: 489
 }),
 'test': Dataset({
     features: ['id', 'context', 'question', 'answer', 'table', 'answer_steps', 'num_steps'],
     num_rows: 616
 })}

Loading Model

In [73]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [74]:
import getpass

# Prompt for the Hugging Face token
hf_token = getpass.getpass("Enter your Hugging Face token: ")

import os
os.environ['HUGGINGFACE_TOKEN'] = hf_token

Enter your Hugging Face token: ··········


In [ ]:
# model_id =  "NousResearch/Llama-2-7b-hf"
model_id = "meta-llama/Llama-2-7b-chat-hf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto",use_auth_token=hf_token)

tokenizer = AutoTokenizer.from_pretrained(model_id,use_auth_token=hf_token)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2193: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]